# Step 2 - Deploy del modelo

## Seteos iniciales

Logueamos sólo los mensajes de warning y error

In [2]:
import logging

logging.getLogger("sagemaker.config").setLevel(logging.WARNING)
logging.getLogger("sagemaker.experiments.run").setLevel(logging.WARNING)

Mostramos las versiones de las librerías de Python importantes para el proyecto

In [3]:
import awscli
import boto3
import numpy
import pandas
import sagemaker

print("sagemaker\t", sagemaker.__version__)
print("pandas\t\t", pandas.__version__)
print("numpy\t\t", numpy.__version__)
print("boto3\t\t", boto3.__version__)
print("awscli\t\t", awscli.__version__)

sagemaker	 2.215.0
pandas		 2.2.2
numpy		 1.26.4
boto3		 1.34.84
awscli		 1.32.84


In [9]:
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sage_session = sagemaker.Session()
bucket_name = sage_session.default_bucket()
prefix = "australia-rain-training"
project_fd = f"s3://{bucket_name}/{prefix}"

print("Region:", region)
print("Rol:", role)
print("Info S3:")
print(f"- Bucket: {bucket_name}")
print(f"- Prefix: {prefix}")

Region: us-east-1
Rol: arn:aws:iam::725128335322:role/service-role/AmazonSageMaker-ExecutionRole-20241121T193663
Info S3:
- Bucket: sagemaker-us-east-1-725128335322
- Prefix: australia-rain-training


## Modelo

In [10]:
import boto3

def get_latest_training_job_name(base_job_name: str) -> str:
    client = boto3.client("sagemaker")
    response = client.list_training_jobs(
        NameContains=base_job_name,
        SortBy="CreationTime", 
        SortOrder="Descending",
        StatusEquals="Completed",
    )
    if response["TrainingJobSummaries"]:
        return response["TrainingJobSummaries"][0]["TrainingJobName"]
    else:
        raise Exception("Training job not found.")

def get_training_job_s3_model_artifacts(job_name: str):
    client = boto3.client("sagemaker")
    response = client.describe_training_job(TrainingJobName=job_name)
    s3_model_artifacts = response["ModelArtifacts"]["S3ModelArtifacts"]
    return s3_model_artifacts

In [11]:
train_base_job_name   = "sagemaker-xgboost-241121-2359-019-481108a1"  # best model of the HPO

latest_train_job_name = get_latest_training_job_name(train_base_job_name)
model_path            = get_training_job_s3_model_artifacts(latest_train_job_name)

print(f"Model path: {model_path}")

Model path: s3://sagemaker-us-east-1-725128335322/australia-rain-training/output/sagemaker-xgboost-241121-2359-019-481108a1/output/model.tar.gz


In [12]:
import time
from sagemaker.xgboost import XGBoostModel

model_name = f"{train_base_job_name}-model-{int(time.time())}"
print("Model name:", model_name)

code_location = f"{project_fd}/code"
xgboost_model = XGBoostModel(
    name=model_name,
    model_data=model_path,
    entry_point="inference.py",
    source_dir="scripts/inference/",
    code_location=code_location,
    framework_version="0.90-2",
    py_version="py3",
    role=role, 
    sagemaker_session=sage_session,
)

Model name: sagemaker-xgboost-241121-2359-019-481108a1-model-1732313087


## Deploy del modelo

In [13]:
import time

from sagemaker.model_monitor import DataCaptureConfig

s3_capture_upload_path = f"{project_fd}/monitoring/datacapture"
print(f"The endpoint will upload captured data to {s3_capture_upload_path}")

endpoint_name = f"{prefix}-sm-endpoint-{int(time.time())}"

print(f"\n*** Endpoint Name ***\n\n{endpoint_name}")

tried_deploying = False

The endpoint will upload captured data to s3://sagemaker-us-east-1-725128335322/australia-rain-training/monitoring/datacapture

*** Endpoint Name ***

australia-rain-training-sm-endpoint-1732313088


In [14]:
assert not tried_deploying, "Se debe volver a crear el endpoint_name antes de intentar deployar de nuevo"
tried_deploying = True

xgboost_model.deploy(
    initial_instance_count=1, 
    instance_type="ml.m5.xlarge", 
    endpoint_name=endpoint_name,
    data_capture_config=DataCaptureConfig(
        enable_capture=True,
        sampling_percentage=100,
        destination_s3_uri=s3_capture_upload_path,
    ),
)

------!

In [56]:
# # ! USAR ESTE CODIGO SOLO SI HUBO UN PROBLEMA CON EL DEPLOY (descomentarlo primero)
# import boto3
# sagemaker = boto3.client("sagemaker")
# sagemaker.delete_endpoint(EndpointName=endpoint_name)

## Inferencia

Probamos la inferencia primero con el split de validation

In [15]:
from sagemaker.deserializers import CSVDeserializer
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer

predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sage_session,
    serializer=CSVSerializer(),
    deserializer=CSVDeserializer(),
)

In [16]:
df_val = pandas.read_csv("australia-rain-all-splits-dbjob_21Nov2024_1732229028468_part00003.csv")
mask_rain = df_val.RainTomorrow > 0.5
df_val

,RainTomorrow,MinTemp,MaxTemp,Rainfall,WindGustDir_north,WindGustDir_east,WindGustSpeed,WindDir9am_north,WindDir9am_east,WindDir3pm_north,WindDir3pm_east,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday
0,0.0,-1.595269,-0.921045,-0.444414,0.627146,-1.302349,-1.085177,0.533982,-1.400132,0.065531,-1.384470,0.134972,-0.419523,0.699739,-0.163140,-2.401607e-12,-5.598720e-13,-1.358776,-0.796732,-0.534165
1,0.0,1.276849,1.999734,-0.444414,-0.491564,1.359029,0.113304,-0.562222,1.345125,-1.256794,0.567784,0.134972,-0.660330,-1.131137,-1.318326,-4.419847e-01,-5.742075e-01,1.126469,2.143901,-0.534165
2,1.0,-0.684980,-1.765780,3.058908,-1.282611,0.579530,1.391685,-1.337355,0.541058,-1.256794,0.567784,1.449489,1.506932,0.430492,1.088311,1.030931e+00,1.022369e+00,-1.187916,-1.752437,1.872072
3,0.0,-0.418172,-1.035585,-0.444414,-0.965762,-0.990125,-0.206291,-1.337355,-0.596065,-1.365744,0.027461,0.612978,-0.178716,-0.754192,-0.066875,1.235097e+00,1.403204e+00,-0.659801,-1.134905,-0.534165
4,0.0,-0.496645,-0.878092,-0.444414,-0.491564,-1.302349,1.311786,0.998640,-1.078068,0.065531,-1.384470,0.373975,1.506932,-0.377247,-1.173928,-2.378182e-01,-2.812577e-01,-0.908325,-0.752622,-0.534165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13546,0.0,-0.324004,0.925918,-0.444414,0.627146,1.359029,-0.525886,-0.562222,-1.400132,1.077595,-0.970925,-1.179545,-0.660330,-0.323398,-1.222061,3.600980e-01,1.142246e-01,-0.395744,0.747101,-0.534165
13547,0.0,-0.936095,-1.479429,3.355800,-1.282611,-0.522850,1.551482,-1.446379,-0.027504,-1.365744,0.027461,0.373975,-0.419523,0.322794,0.943913,5.496811e-01,1.139549e+00,-1.048121,-1.634812,1.872072
13548,0.0,0.915872,0.968871,0.089991,1.101344,1.046805,-0.925380,1.418139,-0.027504,0.613256,1.331916,-0.582037,-0.660330,0.753588,-0.211273,5.496811e-01,2.460520e-01,0.458560,1.041164,1.872072
13549,0.0,-0.418172,1.384080,-0.444414,1.101344,-0.990125,0.512798,-0.014120,1.458219,1.496806,0.027461,-1.418548,0.182494,0.807437,-1.318326,-6.899012e-01,-1.189402e+00,-0.209350,1.394040,-0.534165


In [17]:
print("RAIN TOMORROW")
for rid, row in df_val[mask_rain].iloc[:10].iterrows():
    payload = ",".join([str(v) for v in row])
    print(f"[rid={rid}]\tPrediction:", predictor.predict(payload))
print()
print()

print("NO RAIN TOMORROW")
for rid, row in df_val[~mask_rain].iloc[:10].iterrows():
    payload = ",".join([str(v) for v in row])
    print(f"[rid={rid}]\tPrediction:", predictor.predict(payload))

RAIN TOMORROW
[rid=2]	Prediction: [['no']]
[rid=12]	Prediction: [['yes']]
[rid=13]	Prediction: [['no']]
[rid=14]	Prediction: [['yes']]
[rid=16]	Prediction: [['no']]
[rid=17]	Prediction: [['yes']]
[rid=22]	Prediction: [['yes']]
[rid=26]	Prediction: [['no']]
[rid=31]	Prediction: [['yes']]
[rid=34]	Prediction: [['no']]


NO RAIN TOMORROW
[rid=0]	Prediction: [['no']]
[rid=1]	Prediction: [['no']]
[rid=3]	Prediction: [['no']]
[rid=4]	Prediction: [['no']]
[rid=5]	Prediction: [['no']]
[rid=6]	Prediction: [['no']]
[rid=7]	Prediction: [['no']]
[rid=8]	Prediction: [['no']]
[rid=9]	Prediction: [['no']]
[rid=10]	Prediction: [['no']]


### Data capturada

Dejamos un delay de 2 minutos para que S3 pueda recibir la data

In [18]:
import time
for i in range(12):
    print(f"{i * 10} segundos...")
    time.sleep(10) 
print("LISTO")

0 segundos...
10 segundos...
20 segundos...
30 segundos...
40 segundos...
50 segundos...
60 segundos...
70 segundos...
80 segundos...
90 segundos...
100 segundos...
110 segundos...
LISTO


In [19]:
s3_client = boto3.Session().client("s3")
current_endpoint_capture_prefix = f"{prefix}/monitoring/datacapture/{endpoint_name}"

result = s3_client.list_objects(Bucket=bucket_name, Prefix=current_endpoint_capture_prefix)
capture_files = [
    f"s3://{bucket_name}/{capture_file.get('Key')}"
    for capture_file in result.get('Contents')
]

print("Capture Files: ")
print("\n".join(capture_files))

Capture Files: 
s3://sagemaker-us-east-1-725128335322/australia-rain-training/monitoring/datacapture/australia-rain-training-sm-endpoint-1732313088/AllTraffic/2024/11/22/22/16-11-481-8aab7811-3501-41c6-919f-030e08948087.jsonl


In [21]:
!aws s3 cp {capture_files[0]} datacapture/captured_data_example_0.jsonl
# !head datacapture/captured_data_example.jsonl

download: s3://sagemaker-us-east-1-725128335322/australia-rain-training/monitoring/datacapture/australia-rain-training-sm-endpoint-1732313088/AllTraffic/2024/11/22/22/16-11-481-8aab7811-3501-41c6-919f-030e08948087.jsonl to datacapture/captured_data_example_0.jsonl


Analizamos las primeras 4 predicciones, que dieron no, yes, no, yes respectivamente

In [22]:
import json
with open("datacapture/captured_data_example_0.jsonl", "r") as f:
    data = f.read()

print(json.dumps(json.loads(data.split("\n")[0]), indent=2))
print(json.dumps(json.loads(data.split("\n")[1]), indent=2))
print(json.dumps(json.loads(data.split("\n")[2]), indent=2))
print(json.dumps(json.loads(data.split("\n")[3]), indent=2))

{
  "captureData": {
    "endpointInput": {
      "observedContentType": "text/csv",
      "mode": "INPUT",
      "data": "1.0,-0.6849804950620024,-1.7657795245816,3.058907999752675,-1.2826110910323447,0.5795295544162744,1.391684587712713,-1.337354893312643,0.5410578996955488,-1.2567944839451934,0.5677839595936083,1.4494893845247132,1.5069318362449715,0.4304924195242142,1.0883114516830452,1.0309307402876378,1.022369118095102,-1.1879156230626622,-1.7524374639940354,1.872072001761355",
      "encoding": "CSV"
    },
    "endpointOutput": {
      "observedContentType": "text/csv; charset=utf-8",
      "mode": "OUTPUT",
      "data": "no",
      "encoding": "CSV"
    }
  },
  "eventMetadata": {
    "eventId": "74148be9-d909-46fa-9304-cc18850acb89",
    "inferenceTime": "2024-11-22T22:16:11Z"
  },
  "eventVersion": "0"
}
{
  "captureData": {
    "endpointInput": {
      "observedContentType": "text/csv",
      "mode": "INPUT",
      "data": "1.0,-1.5167962142710103,-0.9926322748846588,-0.38

### Testing

In [24]:
df_test = pandas.read_csv("australia-rain-all-splits-dbjob_21Nov2024_1732229028468_part00002.csv")
mask_rain_test = df_test.RainTomorrow > 0.5
df_test

,RainTomorrow,MinTemp,MaxTemp,Rainfall,WindGustDir_north,WindGustDir_east,WindGustSpeed,WindDir9am_north,WindDir9am_east,WindDir3pm_north,WindDir3pm_east,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday
0,1.0,-0.167058,-0.248120,-0.444414,-1.282611,-0.522850,0.672596,-0.014120,-1.513226,0.613256,1.331916,-0.104031,-1.503153,0.215095,-0.115007,3.163480e-01,3.632320e-01,-0.644268,-0.296824,-0.534165
1,0.0,1.622131,1.112046,-0.444414,0.627146,1.359029,-0.765582,1.309115,-0.596065,0.613256,1.331916,-1.418548,0.664108,-0.054151,0.414453,-9.961509e-01,-1.013632e+00,1.763314,1.173492,-0.534165
2,0.0,1.402406,0.381852,0.624396,-0.491564,1.359029,0.672596,-0.562222,1.345125,-0.482194,1.331916,2.046997,1.506932,1.130533,0.655117,3.926489e-02,2.606995e-01,0.676019,0.423631,1.872072
3,0.0,1.920329,1.154999,0.446261,-1.282611,0.579530,-1.085177,-1.337355,0.541058,1.077595,1.025847,0.612978,-0.419523,1.184383,0.943913,-1.244067e+00,-1.306582e+00,1.235199,1.320524,1.872072
4,1.0,-2.285833,-1.264666,-0.444414,1.101344,-0.990125,0.113304,0.533982,1.345125,0.613256,-1.276993,-1.179545,0.182494,-0.000302,-0.211273,-3.399015e-01,-6.620925e-01,-1.653899,-1.355452,-0.534165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13578,0.0,-0.135668,-0.233803,-0.444414,-1.393874,0.028340,-0.046493,0.533982,-1.400132,-0.482194,1.331916,-0.343034,-0.419523,-1.400384,-0.548202,7.246810e-01,7.440668e-01,-0.100621,-0.179199,-0.534165
13579,1.0,-1.140125,-1.250348,-0.444414,-0.491564,-1.302349,-1.564570,-1.337355,-0.596065,0.065531,0.027461,-0.582037,-2.225574,0.592040,0.751382,1.439264e+00,1.315319e+00,-1.172383,-1.179014,-0.534165
13580,0.0,0.288087,0.424804,-0.444414,-1.282611,-0.522850,0.273102,-1.026880,1.023061,-1.365744,0.027461,0.373975,-0.178716,-0.808041,-0.933264,9.759817e-02,9.957712e-02,0.210035,0.585366,-0.534165
13581,1.0,0.492117,0.453439,0.446261,1.101344,-0.990125,-0.206291,1.309115,-0.596065,1.496806,0.027461,-1.418548,0.062091,0.322794,0.510718,-2.401607e-12,-5.598720e-13,0.489625,0.320709,1.872072


Usamos el endpoint para predecir en test

In [25]:
df_test.loc[:, "RainTomorrow_pred"] = None
print("Total requests run:")
for rid, row in df_test.iterrows():
    if (rid % 1000) == 0:
        print()
    if (rid % 100) == 0:
        print(rid, end="...")
    payload = ",".join([str(v) for v in row])
    df_test.at[rid, "RainTomorrow_pred"] = predictor.predict(payload)[0][0]

print()
print()
print("LISTO")

Total requests run:

0...100...200...300...400...500...600...700...800...900...
1000...1100...1200...1300...1400...1500...1600...1700...1800...1900...
2000...2100...2200...2300...2400...2500...2600...2700...2800...2900...
3000...3100...3200...3300...3400...3500...3600...3700...3800...3900...
4000...4100...4200...4300...4400...4500...4600...4700...4800...4900...
5000...5100...5200...5300...5400...5500...5600...5700...5800...5900...
6000...6100...6200...6300...6400...6500...6600...6700...6800...6900...
7000...7100...7200...7300...7400...7500...7600...7700...7800...7900...
8000...8100...8200...8300...8400...8500...8600...8700...8800...8900...
9000...9100...9200...9300...9400...9500...9600...9700...9800...9900...
10000...10100...10200...10300...10400...10500...10600...10700...10800...10900...
11000...11100...11200...11300...11400...11500...11600...11700...11800...11900...
12000...12100...12200...12300...12400...12500...12600...12700...12800...12900...
13000...13100...13200...13300...13400.

In [26]:
df_test["RainTomorrow"] = df_test["RainTomorrow"].map(lambda v: bool(int(v)))

In [27]:
df_test["RainTomorrow_pred"] = df_test["RainTomorrow_pred"].map(lambda v: True if v == "yes" else v)
df_test["RainTomorrow_pred"] = df_test["RainTomorrow_pred"].map(lambda v: False if v == "no" else v)

In [28]:
df_test[["RainTomorrow", "RainTomorrow_pred"]].value_counts().sort_index()

RainTomorrow  RainTomorrow_pred
False         False                8948
              True                 1657
True          False                1890
              True                 1088
Name: count, dtype: int64

In [29]:
df_test.index.name = "indice"

In [30]:
df_test

,RainTomorrow,MinTemp,MaxTemp,Rainfall,WindGustDir_north,WindGustDir_east,WindGustSpeed,WindDir9am_north,WindDir9am_east,WindDir3pm_north,...,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow_pred
indice,,,,,,,,,,,,,,,,,,,,,
0,True,-0.167058,-0.248120,-0.444414,-1.282611,-0.522850,0.672596,-0.014120,-1.513226,0.613256,...,-0.104031,-1.503153,0.215095,-0.115007,3.163480e-01,3.632320e-01,-0.644268,-0.296824,-0.534165,False
1,False,1.622131,1.112046,-0.444414,0.627146,1.359029,-0.765582,1.309115,-0.596065,0.613256,...,-1.418548,0.664108,-0.054151,0.414453,-9.961509e-01,-1.013632e+00,1.763314,1.173492,-0.534165,True
2,False,1.402406,0.381852,0.624396,-0.491564,1.359029,0.672596,-0.562222,1.345125,-0.482194,...,2.046997,1.506932,1.130533,0.655117,3.926489e-02,2.606995e-01,0.676019,0.423631,1.872072,False
3,False,1.920329,1.154999,0.446261,-1.282611,0.579530,-1.085177,-1.337355,0.541058,1.077595,...,0.612978,-0.419523,1.184383,0.943913,-1.244067e+00,-1.306582e+00,1.235199,1.320524,1.872072,True
4,True,-2.285833,-1.264666,-0.444414,1.101344,-0.990125,0.113304,0.533982,1.345125,0.613256,...,-1.179545,0.182494,-0.000302,-0.211273,-3.399015e-01,-6.620925e-01,-1.653899,-1.355452,-0.534165,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13578,False,-0.135668,-0.233803,-0.444414,-1.393874,0.028340,-0.046493,0.533982,-1.400132,-0.482194,...,-0.343034,-0.419523,-1.400384,-0.548202,7.246810e-01,7.440668e-01,-0.100621,-0.179199,-0.534165,False
13579,True,-1.140125,-1.250348,-0.444414,-0.491564,-1.302349,-1.564570,-1.337355,-0.596065,0.065531,...,-0.582037,-2.225574,0.592040,0.751382,1.439264e+00,1.315319e+00,-1.172383,-1.179014,-0.534165,False
13580,False,0.288087,0.424804,-0.444414,-1.282611,-0.522850,0.273102,-1.026880,1.023061,-1.365744,...,0.373975,-0.178716,-0.808041,-0.933264,9.759817e-02,9.957712e-02,0.210035,0.585366,-0.534165,False


In [31]:
df_test.isnull().sum(axis=0)

RainTomorrow         0
MinTemp              0
MaxTemp              0
Rainfall             0
WindGustDir_north    0
WindGustDir_east     0
WindGustSpeed        0
WindDir9am_north     0
WindDir9am_east      0
WindDir3pm_north     0
WindDir3pm_east      0
WindSpeed9am         0
WindSpeed3pm         0
Humidity9am          0
Humidity3pm          0
Pressure9am          0
Pressure3pm          0
Temp9am              0
Temp3pm              0
RainToday            0
RainTomorrow_pred    0
dtype: int64

In [32]:
df_test.to_csv("australia-rain-test-predicted.csv")